In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

expected_returns = pd.read_excel("Try.xlsx", header=None).values.flatten()
cov_matrix = pd.read_excel(r"E:\大三\大三上\Optimization in Finance\Final Project\Portefolio\Covariance\Covariance.xlsx", header=None).values  # 确保是 N×N

N = len(expected_returns)
print(f"股票数量: {N}")
assert cov_matrix.shape == (N, N), "协方差矩阵维度不匹配！"

股票数量: 499


In [21]:
rf = 0.2

# === 优化变量 ===
w = cp.Variable(N)

# === 目标：最小化方差 ===
objective = cp.Minimize(cp.quad_form(w, cov_matrix))

# === 约束 ===
constraints = [
    expected_returns @ w == 1,   # 固定组合收益为 1（因 rf=0，即超额收益=1）
    w <=0.2                  # 每只股票至少配 0.1%（防止全仓少数股）
    # 注意：不加 sum(w)==1！
]
# 求解
prob = cp.Problem(objective, constraints)
print("正在求解...")

try:
    prob.solve(solver=cp.SCS, verbose=False, eps=1e-6)
except Exception as e:
    print("⚠️ SCS 求解失败，尝试 OSQP...")
    prob.solve(solver=cp.OSQP, verbose=False)

if w.value is not None and prob.status in ["optimal", "optimal_inaccurate"]:
    raw_weights = w.value
    weights = raw_weights / np.sum(raw_weights)  # 归一化到 sum=1

    # 计算最终指标
    portfolio_return = expected_returns @ weights
    portfolio_risk = np.sqrt(weights @ cov_matrix @ weights)
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk

    print("\n" + "="*50)
    print("📈 最优投资组合（最大夏普比率）")
    print("="*50)
    print(f"预期月度收益率: {portfolio_return:.4f} ({portfolio_return*100:.2f}%)")
    print(f"月度波动率（风险）: {portfolio_risk:.4f} ({portfolio_risk*100:.2f}%)")
    print(f"夏普比率: {sharpe_ratio:.4f}")
    print(f"非零持仓股票数: {np.sum(weights > 1e-4)} / {N}")

    # 显示前10大持仓
    top_k = 10
    top_indices = np.argsort(-weights)[:top_k]
    print(f"\n🔝 前 {top_k} 大持仓:")
    print("-" * 30)
    for i in top_indices:
        if weights[i] > 1e-4:
            print(f"股票 {i+1:3d}: {weights[i]:.4f} ({weights[i]*100:6.2f}%)")

    # 保存完整权重
    result_df = pd.DataFrame({
        "Stock_ID": [f"Stock_{i+1}" for i in range(N)],
        "Weight": weights
    })
    result_df.to_csv("optimal_portfolio_weights.csv", index=False)
    print(f"\n💾 完整权重已保存至: optimal_portfolio_weights.csv")

else:
    print("❌ 优化失败！")
    print("求解器状态:", prob.status)
    print("可能原因：数据异常、协方差矩阵不正定、无可行解")

正在求解...

📈 最优投资组合（最大夏普比率）
预期月度收益率: 0.0543 (5.43%)
月度波动率（风险）: 0.0000 (0.00%)
夏普比率: -22280656.7077
非零持仓股票数: 342 / 499

🔝 前 10 大持仓:
------------------------------
股票  55: 0.0107 (  1.07%)
股票 391: 0.0104 (  1.04%)
股票 434: 0.0103 (  1.03%)
股票 360: 0.0103 (  1.03%)
股票 497: 0.0102 (  1.02%)
股票 457: 0.0101 (  1.01%)
股票  46: 0.0100 (  1.00%)
股票 133: 0.0100 (  1.00%)
股票  64: 0.0099 (  0.99%)
股票 381: 0.0099 (  0.99%)

💾 完整权重已保存至: optimal_portfolio_weights.csv


In [18]:
N = len(expected_returns)  # 或直接 len(cov_matrix)
min_weight = 0.01  # 最小持仓 1%
M = 1.0            # 大数上界
target_return = 0.01

# === 变量 ===
w = cp.Variable(N)          # 连续权重
z = cp.Variable(N, boolean=True)  # 0-1 变量：是否持仓

# === 目标：最小化组合方差（即最小化风险）===
objective = cp.Minimize(cp.quad_form(w, cov_matrix))

# === 约束 ===
constraints = [
    cp.sum(w) == 1.0,        # 权重和为 1（满仓）
    w >= 0,
    w<=0.1,
    cp.sum(z) <=30,
    expected_returns @ w >= target_return,
    w >= min_weight * z,    
    w <= M * z,          
]

# === 求解（使用支持 MIP 的求解器）===
prob = cp.Problem(objective, constraints)

print("正在求解全局最小方差组合（带最小持仓约束）...")
try:
    prob.solve(solver=cp.ECOS_BB, verbose=True, max_iters=5000)
except Exception as e:
    print("ECOS_BB 求解失败:", e)
    raise

if w.value is not None and prob.status in ["optimal", "optimal_inaccurate"]:
    weights = w.value

    weights = np.maximum(weights, 0)
    weights = weights / np.sum(weights) 

    portfolio_risk = np.sqrt(weights @ cov_matrix @ weights)
    portfolio_return = expected_returns @ weights 

    rf = 0.005
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk

    print(f"\n✅ 全局最小方差组合（GMV）结果:")
    print(f"组合风险（标准差）: {portfolio_risk:.6f} ({portfolio_risk*100:.3f}%)")
    print(f"组合预期收益: {portfolio_return:.6f} ({portfolio_return*100:.3f}%)")
    print(f"夏普比率: {sharpe_ratio:.4f}")  
    print(f"持仓股票数量: {np.sum(weights > 1e-4)} / {N}")

    # 输出非零权重（≥1% 的）
    nonzero_indices = np.where(weights > 1e-4)[0]
    print(f"\n📈 非零持仓（共 {len(nonzero_indices)} 只）:")
    for i in nonzero_indices:
        print(f"  股票 {i+1:3d}: {weights[i]:.4f} ({weights[i]*100:6.2f}%)")

    # 保存结果
    import pandas as pd
    result_df = pd.DataFrame({
        "Stock_ID": [f"Stock_{i+1}" for i in range(N)],
        "Weight": weights
    })
    result_df.to_csv("min_variance_portfolio_with_min_position.csv", index=False)
    print("\n💾 权重已保存至 'min_variance_portfolio_with_min_position.csv'")

else:
    print("❌ 优化失败！")
    print("状态:", prob.status)
    print("可能原因：N 太大、求解器能力不足、问题无可行解")

(CVXPY) Nov 27 09:46:17 AM: Your problem has 998 variables, 1999 constraints, and 0 parameters.


(CVXPY) Nov 27 09:46:17 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 27 09:46:17 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 27 09:46:17 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 27 09:46:17 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 27 09:46:17 AM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Nov 27 09:46:17 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS_BB
(CVXPY) Nov 27 09:46:17 AM: Applying reduction Dcp2Cone
(CVXPY) Nov 27 09:46:17 AM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 27 09:46:17 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Nov 27 09:46:17 AM: Applying reduction ECOS_BB
(CVXPY) Nov 27 09:46:17 AM: Finished problem compilation (took 7.392e-02 seconds).
(CVXPY) Nov 27 09:46:17 AM: Invoking solver ECOS_BB  to obtain a s

正在求解全局最小方差组合（带最小持仓约束）...
                                     CVXPY                                     
                                     v1.7.2                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------


(CVXPY) Nov 27 09:46:18 AM: Problem status: optimal
(CVXPY) Nov 27 09:46:18 AM: Optimal value: 1.353e-05
(CVXPY) Nov 27 09:46:18 AM: Compilation took 7.392e-02 seconds
(CVXPY) Nov 27 09:46:18 AM: Solver (including time spent in interface) took 1.221e-01 seconds


-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------

✅ 全局最小方差组合（GMV）结果:
组合风险（标准差）: 0.003679 (0.368%)
组合预期收益: 0.010000 (1.000%)
夏普比率: 1.3591
持仓股票数量: 19 / 499

📈 非零持仓（共 19 只）:
  股票   6: 0.0925 (  9.25%)
  股票   7: 0.0297 (  2.97%)
  股票  23: 0.0226 (  2.26%)
  股票  55: 0.0298 (  2.98%)
  股票  64: 0.0628 (  6.28%)
  股票 133: 0.0266 (  2.66%)
  股票 148: 0.0441 (  4.41%)
  股票 176: 0.0402 (  4.02%)
  股票 189: 0.0259 (  2.59%)
  股票 231: 0.0269 (  2.69%)
  股票 251: 0.0160 (  1.60%)
  股票 309: 0.0582 (  5.82%)
  股票 320: 0.0807 (  8.07%)
  股票 358: 0.0605 (  6.05%)
  股票 376: 0.1000 ( 10.00%)
  股票 408: 0.0643 (  6.43%)
  股票 430: 0.0407 (  4.07%)
  股票 434: 0.0785 (  7.85%)
  股票 454: 0.1000 ( 10.00%)

💾 权重已保存至 'min_variance_portfolio_with_min_position.csv'
